In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style
from webdriver_manager.chrome import ChromeDriverManager




print(plt.style.available)
%matplotlib inline
driver = webdriver.Chrome()

url='https://cq.esf.fang.com/house-a060-b014802.html'
driver.get(url)
driver.find_element(By.XPATH, '/html/body/div[4]/div[4]').click()

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'petroff10', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [5]:
all_data = [] 
page_num = 1

while True:
    print(f"\n--- 正在处理第 {page_num} 页 ---")
    
    # 抓取当前页的房源数据
    try:
        # 内部找所有 <dl>
        list_container = driver.find_element(By.CSS_SELECTOR,'.shop_list.shop_list_4')
        all_house_items = list_container.find_elements(By.TAG_NAME,'dl')
        print(f"本页找到 {len(all_house_items)} 个潜在房源条目。")
        
        # 遍历所有潜在的房源条目
        for item in all_house_items:
            try:
                # 尝试抓取关键字段：如果任何一个字段不存在，就触发异常，跳过这个房源
                title = item.find_element(By.CLASS_NAME, 'tel_shop').text
                price = item.find_element(By.CLASS_NAME, 'red').text
                # 抓取成功，创建一行数据并加入总列表
                row_data = [title, price]
                all_data.append(row_data)
                
            except NoSuchElementException:
                pass # 忽略不完整的条目，继续下一个
                
    except NoSuchElementException:
        print("致命错误：无法定位房源列表容器。停止抓取。")
        break
        
    #尝试翻页 (必须在数据抓取之后)
    try:
        next_page_xpath = "//div[@class='page_box']//a[text()='下一页']"
        next_page_button = driver.find_element(By.XPATH, next_page_xpath)
        # 执行点击操作
        next_page_button.click()
       
        page_num += 1
        
    except NoSuchElementException:
        # 如果找不到“下一页”按钮，说明是最后一页，退出循环
        print("已到达最后一页，结束抓取。")
        break
        
    except Exception as e:
        print(f"翻页过程中发生其他错误: {e}")
        break


# 最终将所有数据转换为 DataFrame
df = pd.DataFrame(all_data, columns=['Title', 'Price']) 
print("\n数据抓取完成。最终 DataFrame 已创建。")


--- 正在处理第 1 页 ---
本页找到 60 个潜在房源条目。

--- 正在处理第 2 页 ---
本页找到 60 个潜在房源条目。

--- 正在处理第 3 页 ---
本页找到 60 个潜在房源条目。

--- 正在处理第 4 页 ---
本页找到 60 个潜在房源条目。

--- 正在处理第 5 页 ---
本页找到 19 个潜在房源条目。
已到达最后一页，结束抓取。

数据抓取完成。最终 DataFrame 已创建。


In [6]:
df

,Title,Price
0,4室2厅 | 121.91㎡ |中层 （共32层） | 东向 | 2012年建 |汤宁,59万
1,2室2厅 | 69.88㎡ |中层 （共18层） | 南北向 | 2012年建 |左其亮,38万
2,4室2厅 | 121.33㎡ |中层 （共26层） | 南向 | 2012年建 |李采莹,50万
3,3室2厅 | 98㎡ |中层 （共16层） | 西向 | 2012年建 |王小红,65万
4,3室2厅 | 126.64㎡ |中层 （共16层） | 西向 | 2012年建 |王小红,80万
...,...,...
254,3室2厅 | 102.56㎡ |中层 （共26层） | 北向,57万
255,2室2厅 | 78.48㎡ |中层 （共18层） | 东向,54万
256,122.18㎡,43.1万
257,9㎡,35.16万


In [7]:
df['Size'] = df['Title'].astype(str).str.extract(r'(\d+\.?\d*)\s*㎡')

In [8]:
df

,Title,Price,Size
0,4室2厅 | 121.91㎡ |中层 （共32层） | 东向 | 2012年建 |汤宁,59万,121.91
1,2室2厅 | 69.88㎡ |中层 （共18层） | 南北向 | 2012年建 |左其亮,38万,69.88
2,4室2厅 | 121.33㎡ |中层 （共26层） | 南向 | 2012年建 |李采莹,50万,121.33
3,3室2厅 | 98㎡ |中层 （共16层） | 西向 | 2012年建 |王小红,65万,98
4,3室2厅 | 126.64㎡ |中层 （共16层） | 西向 | 2012年建 |王小红,80万,126.64
...,...,...,...
254,3室2厅 | 102.56㎡ |中层 （共26层） | 北向,57万,102.56
255,2室2厅 | 78.48㎡ |中层 （共18层） | 东向,54万,78.48
256,122.18㎡,43.1万,122.18
257,9㎡,35.16万,9


In [9]:
columns_to_drop = ['Title'] 
df_final = df.drop(columns=columns_to_drop, axis=1)
df_final

,Price,Size
0,59万,121.91
1,38万,69.88
2,50万,121.33
3,65万,98
4,80万,126.64
...,...,...
254,57万,102.56
255,54万,78.48
256,43.1万,122.18
257,35.16万,9


In [10]:
df_final['Size'] = pd.to_numeric(df_final['Size'], errors='coerce')
df_filtered = df_final[df_final['Size'] >= 25].copy()
df2=df_filtered
df2

,Price,Size
0,59万,121.91
1,38万,69.88
2,50万,121.33
3,65万,98.00
4,80万,126.64
...,...,...
252,60.7万,103.90
253,68万,104.40
254,57万,102.56
255,54万,78.48


In [11]:
df2['总价(万)'] = df2['Price'].astype(str).str.replace('万', '', regex=False)
df2['总价(万)'] = pd.to_numeric(df2['总价(万)'], errors='coerce')
df2['面积(㎡)'] = pd.to_numeric(df2['Size'], errors='coerce')
df2['单价(元/㎡)'] = df2['总价(万)'] *10000/ df2['面积(㎡)']
df2['单价(元/㎡)'] = df2['单价(元/㎡)'].round(0)
df_final2 = df2.drop(columns=['Price','Size'], axis=1).copy()
df_final2

,总价(万),面积(㎡),单价(元/㎡)
0,59.0,121.91,4840.0
1,38.0,69.88,5438.0
2,50.0,121.33,4121.0
3,65.0,98.00,6633.0
4,80.0,126.64,6317.0
...,...,...,...
252,60.7,103.90,5842.0
253,68.0,104.40,6513.0
254,57.0,102.56,5558.0
255,54.0,78.48,6881.0


In [26]:
df_final2.to_excel('陈家桥售价数据.xlsx', index=False)